In [1]:
from scadaUtils import comUtils
import os,sys,re, pandas as pd

# 1.Build the configuration of your project using the Conf_generator
Let's say we have the following configuration.

In [ ]:
dummy_df_devices = pd.read_excel('data/dummy_devices.ods', sheet_name = 'devices', index_col=0)
dummy_df_devices

We load the modbus tables.

In [ ]:
dummy_modbus_map1 = pd.read_excel('data/dummy_devices.ods', sheet_name = 'dummy1', index_col = 0)
dummy_modbus_map2 = pd.read_excel('data/dummy_devices.ods', sheet_name = 'dummy2', index_col = 0)
dummy_modbus_map1

# 2. Start the project with this configuration of devices

In [ ]:
def generate_dummy_conf():
    return {
        'df_devices': dummy_df_devices,
        'modbus_maps' : {
            'dummy1':dummy_modbus_map1,
            'dummy2':dummy_modbus_map2
        },
        'tag_categories':{'dummy':{'water line':['PT_L5_H2O.HM05','PT_L7_H2O.HM05']}}
    }

In [ ]:
from scadaUtils import Conf_generator 
conf = Conf_generator.Conf_generator_RT(project_name='dummy_project_gaia', function_generator=generate_dummy_conf,force_creation=True)

In [ ]:
# conf.dfplc

In [ ]:
from scadaUtils import GAIA
# GAIA.GAIA?
gaia = GAIA.GAIA(project_name='dummy_project_gaia', function_generator=generate_dummy_conf,realtime=True,verbose=True,force_creation=True)

# 2. DUMP DATA 

## 2.1 Start the SIMULATOR of the modbus device (optionnal)
As we don't have the dummy device delivering data with the modbus protocole, we will create a simulator of the device based on the same modbus map and we will run the modbus server to serve data (random data). 
For your real project you can of course skip this step if the device is available on your network.

In [ ]:
from scadaUtils.Simulators import SimulatorModeBus

dummy_simulator1 = SimulatorModeBus(
   port = gaia.devices['dummy1'].port,
    modbus_map = gaia.devices['dummy1'].modbus_map,
    simulator_name = 'dummy_gaia_simulator',
    bo = gaia.devices['dummy1'].byte_order,
    wo = gaia.devices['dummy1'].word_order)

dummy_simulator2 = SimulatorModeBus(
   port = gaia.devices['dummy2'].port,
    modbus_map = gaia.devices['dummy2'].modbus_map,
    simulator_name = 'dummy_gaia_simulator',
    bo = gaia.devices['dummy2'].byte_order,
    wo = gaia.devices['dummy2'].word_order)

dummy_simulator1.start()
dummy_simulator2.start()

In [ ]:
# Start dumping data
# param bool park_on_time: if True, it will wait for the parking to start at a round time.
gaia.start_dumping(park_on_time=False)

In [ ]:
gaia.run_GUI(port=19000)

## check Dumping  

In [ ]:
gaia.read_db()

In [ ]:
dummy_gaia.stop_dumping()

## Check loading data

## Test data collection

In [ ]:
device = gaia._dumper.devices['dummy1']

In [ ]:
device.connectDevice()

In [ ]:
tags = device.dfplc.index.to_list()
device.collectData('CET',tags)

In [ ]:
device._client.read_holding_registers('CET',tags)
device.quick_decode_tag('CET',tags)

# quick

In [ ]:
from scadaUtils import comUtils
import warnings
warnings.filterwarnings("ignore")
import os,sys,re, pandas as pd
dummy_df_devices = pd.read_excel('data/dummy_devices.ods', sheet_name = 'devices', index_col=0)
dummy_modbus_map1 = pd.read_excel('data/dummy_devices.ods', sheet_name = 'dummy1', index_col = 0)
dummy_modbus_map2 = pd.read_excel('data/dummy_devices.ods', sheet_name = 'dummy2', index_col = 0)
def generate_dummy_conf():
    return {
        'df_devices': dummy_df_devices,
        'modbus_maps' : {
            'dummy1':dummy_modbus_map1,
            'dummy2':dummy_modbus_map2
        },
        'tag_categories':{'dummy':{'water line':['PT_L5_H2O.HM05','PT_L7_H2O.HM05']}}
    }
from scadaUtils import GAIA
gaia = GAIA.GAIA(project_name='dummy_project_gaia', function_generator=generate_dummy_conf,realtime=True,verbose=True,force_creation=True)

In [ ]:
from scadaUtils.Simulators import SimulatorModeBus
dummy_simulator1 = SimulatorModeBus(
   port = gaia.devices['dummy1'].port,
    modbus_map = gaia.devices['dummy1'].modbus_map,
    simulator_name = 'dummy_gaia_simulator',
    bo = gaia.devices['dummy1'].byte_order,
    wo = gaia.devices['dummy1'].word_order)

dummy_simulator2 = SimulatorModeBus(
   port = gaia.devices['dummy2'].port,
    modbus_map = gaia.devices['dummy2'].modbus_map,
    simulator_name = 'dummy_gaia_simulator',
    bo = gaia.devices['dummy2'].byte_order,
    wo = gaia.devices['dummy2'].word_order)

dummy_simulator1.start()
dummy_simulator2.start()

In [ ]:
gaia.start_dumping(park_on_time=False)

In [ ]:
gaia.run_GUI(port=19000)

In [ ]:
# gaia.read_db()

In [ ]:
t0 = pd.Timestamp('2024-01-15 05:00',tz='CET')
t1 = pd.Timestamp('2024-01-17 14:30',tz='CET')
tags = ['PT_L5_H2O.HM05','TT_L5_H2O.HM05']

In [ ]:
df = gaia.loadtags_period(t0,t1,tags,model='dummy',verbose=True,rs='10s')
df

In [ ]:
gaia.park_database()

## generate random data statically

In [ ]:
import numpy as np
from scadaUtils.comUtils import FORMAT_DAY,create_folder_if_not
# def generate_random_data(self,t0,t1):
self = gaia
t0 = pd.Timestamp('2024-01-15 02:00',tz='CET')
t1 = pd.Timestamp('2024-01-16 13:00',tz='CET')
listdays = [k.strftime(FORMAT_DAY) for k in pd.date_range(t0,t1,ambiguous=True)]
init_values = []
for name,device in self.devices.items():
    init_values.append(device.modbus_map[['minimum','maximum']].apply(lambda x:x[0] + np.random.randint(0,x[1]-x[0]),axis=1))
init_values = pd.concat(init_values)

In [ ]:
for t in init_values.index.to_list():
    idx = pd.date_range(start=t0,end=t1,freq=str(gaia.conf.dfplc.loc[t,'FREQUENCE_ECHANTILLONNAGE'])+'S').tz_convert('UTC')
    s = pd.Series(init_values[t] + np.random.rand(len(idx))*init_values[t]*0.02,index=idx)
    s.index.name='timestampz'
    s.name='value'
    for d in listdays:
        t00 = pd.Timestamp(d + ' 00:00:00',tz='UTC')
        t10 = t0 + pd.Timedelta(days=1)
        ss = s[(s.index>=t00)&(s.index<t10)]
        folderday = os.path.join(gaia.conf.parameters['folderpkl'],'dummy',d)
        create_folder_if_not(folderday)
        namefile = os.path.join(folderday,t + '.pkl')
        ss.to_pickle(namefile)

In [ ]:
pd.read_pickle('/home/dorian/dummy_project_gaia_user/dummy_project_gaia_daily/dummy/2024-01-16/TT_L5_H2O.HM05.pkl')

In [ ]:
gaia.conf.dfplc